In [5]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import re

def scrape_lawson_products_no_pandas():
    base_url = "https://www.lawson.co.jp"
    start_url = "https://www.lawson.co.jp/recommend/index.html"

    print("カテゴリページのリンクを取得中...")
    try:
        response = requests.get(start_url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
    except requests.exceptions.RequestException as e:
        print(f"初期ページの取得中にエラーが発生しました ({start_url}): {e}")
        return []

    category_links_html = soup.find('ul', class_='contentsNav4')
    if not category_links_html:
        print("カテゴリリンクが見つかりません。")
        return []

    category_links = []
    for li in category_links_html.find_all('li'):
        a_tag = li.find('a')
        if a_tag and a_tag.get('href'):
            href = a_tag['href']
            # 「沖縄」のように別ドメインに飛ぶリンクは除外する
            if "okinawa.lawson.jp" in href:
                print(f"警告: 外部ドメインへのリンクをスキップします: {href}")
                continue
            
            # カテゴリページへのリンクであることを確認 (recommend/original/ または recommend/ent/ から始まるもの)
            if not ("/recommend/original/" in href or "/recommend/ent/" in href or "/loppigoods/" in href):
                 print(f"警告: 目的外のカテゴリリンクをスキップします: {href}")
                 continue

            if not href.startswith('http'):
                category_links.append(base_url + href)
            else:
                category_links.append(href)

    all_products = []

    for category_url in category_links:
        print(f"カテゴリページをスクレイピング中: {category_url}")
        try:
            category_response = requests.get(category_url, timeout=10)
            category_response.raise_for_status()
            category_soup = BeautifulSoup(category_response.content, 'html.parser')

            # --- ここから商品リストの取得ロジックを修正 ---
            product_list = []
            
            # productList クラスを持つ article タグを見つける
            # これにより、フッターなどの関係ない ul.col-4.heightLineParent を除外
            product_article = category_soup.find('article', class_='productList')
            
            if product_article:
                # その article タグの中から ul.col-4.heightLineParent を探す
                product_list_containers = product_article.find_all('ul', class_='col-4 heightLineParent')
                for container in product_list_containers:
                    product_list.extend(container.find_all('li'))
            else:
                print(f"警告: カテゴリページ {category_url} で 'article.productList' が見つかりませんでした。直接 ul.col-4.heightLineParent を探します。")
                # もし article.productList がない場合は、以前のようにページ全体から探す（フォールバック）
                product_list_containers = category_soup.find_all('ul', class_='col-4 heightLineParent')
                for container in product_list_containers:
                    product_list.extend(container.find_all('li'))

            if not product_list:
                print(f"警告: カテゴリページ {category_url} で商品リストが見つかりませんでした。スキップします。")
                time.sleep(1)
                continue
            # --- 修正ここまで ---

            for product_block in product_list:
                product_data = {}

                product_url_tag = product_block.find('a')
                if product_url_tag and product_url_tag.get('href'):
                    relative_url = product_url_tag['href']
                    # 詳細ページリンクが外部サイト（例: Loppiグッズ）の場合もあるので、ドメインチェックも入れる
                    if not relative_url.startswith('http') and not relative_url.startswith('//'): # //で始まるURLも対応
                        product_data['url'] = base_url + relative_url
                    elif relative_url.startswith('//'): # //で始まる場合はスキーマを付加
                        product_data['url'] = 'https:' + relative_url
                    else:
                        product_data['url'] = relative_url
                else:
                    product_data['url'] = None
                    # ここで URL が見つからない場合は、このブロックは有効な商品ではない可能性が高いのでスキップ
                    # 例外として、商品リスト内の広告や注意書きなどがliタグで表現されている場合があるため
                    # 警告メッセージをより具体的にする
                    first_few_chars = product_block.get_text(strip=True)[:100].replace('\n', ' ')
                    print(f"警告: 商品詳細URLが見つからないli要素をスキップします。内容の冒頭: '{first_few_chars}'")
                    continue

                product_name_tag = product_block.find('p', class_='ttl')
                product_data['product_name'] = product_name_tag.get_text(strip=True) if product_name_tag else None

                picture_tag = product_block.find('img')
                if picture_tag and picture_tag.get('src'):
                    relative_src = picture_tag['src']
                    if not relative_src.startswith('http') and not relative_src.startswith('//'):
                        product_data['picture'] = base_url + relative_src
                    elif relative_src.startswith('//'):
                        product_data['picture'] = 'https:' + relative_src
                    else:
                        product_data['picture'] = relative_src
                else:
                    product_data['picture'] = None

                price_tag = product_block.find('p', class_='price')
                if price_tag:
                    price_text = price_tag.get_text(strip=True).replace('本体価格', '').replace('円', '').replace('税込', '').strip()
                    product_data['price'] = price_text
                else:
                    product_data['price'] = None
                
                calorie_from_list_tag = product_block.find('p', string=re.compile(r'\d+kcal'))
                if calorie_from_list_tag:
                    calorie_match = re.search(r'(\d+)kcal', calorie_from_list_tag.get_text())
                    if calorie_match:
                        product_data['calorie'] = calorie_match.group(1)
                
                if product_data['url']:
                    # Loppiグッズなど、lawson.co.jp ではない外部URLの場合は詳細ページスクレイピングをスキップ
                    if not product_data['url'].startswith(base_url):
                        print(f"情報: 外部サイトの商品詳細ページ ({product_data['url']}) のスクレイピングはスキップします。")
                        # 栄養成分はNoneのままにする
                    else:
                        try:
                            detail_response = requests.get(product_data['url'], timeout=10)
                            detail_response.raise_for_status()
                            detail_soup = BeautifulSoup(detail_response.content, 'html.parser')

                            nutrition_facts_ul = detail_soup.find('div', class_='nutritionFacts_table')
                            if nutrition_facts_ul:
                                for li_tag in nutrition_facts_ul.find_all('li'):
                                    dt_tag = li_tag.find('dt')
                                    dd_tag = li_tag.find('dd')
                                    if dt_tag and dd_tag:
                                        key = dt_tag.get_text(strip=True)
                                        value = dd_tag.get_text(strip=True).replace('kcal', '').replace('g', '').replace('mg', '').strip()
                                        
                                        if key == '熱量':
                                            product_data['calorie'] = value
                                        elif key == 'たんぱく質':
                                            product_data['protein'] = value
                                        elif key == '脂質':
                                            product_data['fat'] = value
                                        elif key == '炭水化物':
                                            product_data['carbohydrate'] = value
                                        elif key == '糖質': 
                                            product_data['sugar'] = value
                                        elif key == '食物繊維':
                                            product_data['fiber'] = value
                                        elif key == '食塩相当量':
                                            product_data['salt'] = value
                            else:
                                nutrition_table = detail_soup.find('table', class_='nutrition-table')
                                if nutrition_table:
                                    for row in nutrition_table.find_all('tr'):
                                        th = row.find('th')
                                        td = row.find('td')
                                        if th and td:
                                            key = th.get_text(strip=True).replace('：', '')
                                            value = td.get_text(strip=True).replace('kcal', '').replace('g', '').replace('mg', '').strip()
                                            if key == 'エネルギー':
                                                product_data['calorie'] = value
                                            elif key == 'たんぱく質':
                                                product_data['protein'] = value
                                            elif key == '脂質':
                                                product_data['fat'] = value
                                            elif key == '炭水化物':
                                                product_data['carbohydrate'] = value
                                            elif key == '糖質':
                                                product_data['sugar'] = value
                                            elif key == '食物繊維':
                                                product_data['fiber'] = value
                                            elif key == '食塩相当量':
                                                product_data['salt'] = value
                                else:
                                    nutrition_info_div = detail_soup.find('div', class_='text-area')
                                    if nutrition_info_div:
                                        text_content = nutrition_info_div.get_text()
                                        calorie_match = re.search(r'エネルギー[\s\S]*?(\d+\.?\d*)kcal', text_content)
                                        protein_match = re.search(r'たんぱく質[\s\S]*?(\d+\.?\d*)g', text_content)
                                        fat_match = re.search(r'脂質[\s\S]*?(\d+\.?\d*)g', text_content)
                                        carbohydrate_match = re.search(r'炭水化物[\s\S]*?(\d+\.?\d*)g', text_content)
                                        sugar_match = re.search(r'(?:糖質|内\s*糖質)[\s\S]*?(\d+\.?\d*)g', text_content)
                                        fiber_match = re.search(r'(?:食物繊維|内\s*食物繊維)[\s\S]*?(\d+\.?\d*)g', text_content)
                                        salt_match = re.search(r'食塩相当量[\s\S]*?(\d+\.?\d*)g', text_content)

                                        if calorie_match: product_data['calorie'] = calorie_match.group(1)
                                        if protein_match: product_data['protein'] = protein_match.group(1)
                                        if fat_match: product_data['fat'] = fat_match.group(1)
                                        if carbohydrate_match: product_data['carbohydrate'] = carbohydrate_match.group(1)
                                        if sugar_match: product_data['sugar'] = sugar_match.group(1)
                                        if fiber_match: product_data['fiber'] = fiber_match.group(1)
                                        if salt_match: product_data['salt'] = salt_match.group(1)

                        except requests.exceptions.RequestException as e:
                            print(f"商品詳細ページの取得中にエラーが発生しました ({product_data['url']}): {e}")
                
                desired_keys = ['product_name', 'price', 'picture', 'calorie', 'protein', 'fat', 'carbohydrate', 'sugar', 'fiber', 'salt', 'url']
                for key in desired_keys:
                    if key not in product_data:
                        product_data[key] = None

                all_products.append(product_data)
                time.sleep(0.5)

        except requests.exceptions.RequestException as e:
            print(f"カテゴリページの取得中にエラーが発生しました ({category_url}): {e}")
        time.sleep(1)

    return all_products

products_data = scrape_lawson_products_no_pandas()

if products_data:
    fieldnames = ['product_name', 'price', 'picture', 'calorie', 'protein', 'fat', 'carbohydrate', 'sugar', 'fiber', 'salt', 'url']

    with open("lawson_products.csv", "w", newline="", encoding='utf-8-sig') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for product in products_data:
            writer.writerow(product)
    
    print("スクレイピングが完了し、lawson_products.csv に保存されました。")
else:
    print("商品データが取得できませんでした。")

カテゴリページのリンクを取得中...
カテゴリページをスクレイピング中: https://www.lawson.co.jp/recommend/original/rice/
警告: 商品詳細URLが見つからないli要素をスキップします。内容の冒頭: '※熱量表示は関東地域のものを掲載しております。'
警告: 商品詳細URLが見つからないli要素をスキップします。内容の冒頭: '※北海道・東北・沖縄地域のローソンではお取り扱いしておりません。'
警告: 商品詳細URLが見つからないli要素をスキップします。内容の冒頭: '※沖縄地域のローソンではお取り扱いしておりません。'
警告: 商品詳細URLが見つからないli要素をスキップします。内容の冒頭: '※沖縄地域のローソンではお取り扱いしておりません。'
警告: 商品詳細URLが見つからないli要素をスキップします。内容の冒頭: '※沖縄地域のローソン、ナチュラルローソンではお取り扱いしておりません。'
警告: 商品詳細URLが見つからないli要素をスキップします。内容の冒頭: '※北海道地域のローソンではお取り扱いしておりません。'
警告: 商品詳細URLが見つからないli要素をスキップします。内容の冒頭: '※ナチュラルローソンではお取り扱いしておりません。'
警告: 商品詳細URLが見つからないli要素をスキップします。内容の冒頭: '※沖縄地域のローソンではお取り扱いしておりません。'
警告: 商品詳細URLが見つからないli要素をスキップします。内容の冒頭: '※ナチュラルローソンではお取り扱いしておりません。'
警告: 商品詳細URLが見つからないli要素をスキップします。内容の冒頭: '※ナチュラルローソンではお取り扱いしておりません。'
警告: 商品詳細URLが見つからないli要素をスキップします。内容の冒頭: '※ナチュラルローソンではお取り扱いしておりません。'
警告: 商品詳細URLが見つからないli要素をスキップします。内容の冒頭: '※北陸地域のローソンではお取り扱いしておりません。'
警告: 商品詳細URLが見つからないli要素をスキップします。内容の冒頭: '※沖縄地域のローソン、ナチュラルローソンではお取り扱いしておりません。'
警告: 商品詳細URLが見つからないli要素をスキップしま